# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import sqlalchemy as db
from sqlalchemy import create_engine
import io
import psycopg2

# Import Data Preparation Dependency for Algorithms
from sklearn.model_selection import train_test_split

# Import Tree Regressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
from config import db_password, username

# Connect To PostgreSQL Database With SQLAlchemy

In [3]:
host = 'movies-fp.cpige012zhtw.us-east-1.rds.amazonaws.com'
port = 5432
passw = db_password
database = "postgres"
port=5432

   
db_string = "postgresql://" + username + ":" + passw + "@" + host + ":" + "5432/" + database
engine = db.create_engine(db_string)
connection = engine.connect()
metadata = db.MetaData()

# Query Database and Select Tables To Import and Create Dataframes

In [4]:
movie_genres = db.Table('movie_genre', metadata, autoload=True, autoload_with=engine)
query = db.select([movie_genres]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
mov_gen_df = pd.DataFrame(ResultSet)
mov_gen_df.columns = ResultSet[0].keys()
mov_gen_df = mov_gen_df.loc[:, ['movie_id', 'mg_id', 'genre_id']]

mov_gen_df.head()

,movie_id,mg_id,genre_id
0,1,1,1
1,1,2,3
2,1,3,4
3,1,4,5
4,1,5,9


In [5]:
movies = db.Table('movies', metadata, autoload=True, autoload_with=engine)
query = db.select([movies]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
movies_df = pd.DataFrame(ResultSet)
movies_df.columns = ResultSet[0].keys()
movies_df = movies_df.loc[:, ["movie_id", "name", "year", "runtime"]]

movies_df.head()

,movie_id,name,year,runtime
0,1,Toy Story,1995,81
1,2,Jumanji,1995,104
2,3,Grumpier Old Men,1995,101
3,4,Waiting to Exhale,1995,127
4,5,Father of the Bride Part II,1995,106


In [6]:
ratings = db.Table('ratings', metadata, autoload=True, autoload_with=engine)
query = db.select([ratings]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ratings_df = pd.DataFrame(ResultSet)
ratings_df.columns = ResultSet[0].keys()
ratings_df = ratings_df.loc[:, ['movie_id', 'user_id', 'rating']]

ratings_df.head()

,movie_id,user_id,rating
0,1270,96029,4.5
1,1292,96029,3.5
2,1302,96029,4.5
3,1376,96029,4.0
4,1377,96029,3.5


In [7]:
cast = db.Table('movie_cast', metadata, autoload=True, autoload_with=engine)
query = db.select([cast])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
cast_df = pd.DataFrame(ResultSet)
cast_df.columns = ResultSet[0].keys()
cast_df = cast_df.loc[:, ['movie_id', 'cast_id']]

cast_df.head()

,movie_id,cast_id
0,1,31
1,2,2157
2,3,6837
3,4,8851
4,5,67773


In [8]:
connection.close()

In [9]:
def drop_no_genres(column):
    i = -1
    na = float("Nan")
    mov_gen_df["clean_genres"] = ""
    for row in column:
        i += 1
        if row == 20:
            mov_gen_df.at[i, "clean_genres"] = na
        else:
            mov_gen_df.at[i, "clean_genres"] = row
    return mov_gen_df.head()

In [10]:
drop_no_genres(mov_gen_df["genre_id"])

,movie_id,mg_id,genre_id,clean_genres
0,1,1,1,1
1,1,2,3,3
2,1,3,4,4
3,1,4,5,5
4,1,5,9,9


In [11]:
mov_gen_df.count()

movie_id        58098
mg_id           58098
genre_id        58098
clean_genres    57819
dtype: int64

In [12]:
mov_gen_df = mov_gen_df.drop_duplicates(keep="first")
mov_gen_df = mov_gen_df.dropna()

In [13]:
mov_gen_df.count()

movie_id        57819
mg_id           57819
genre_id        57819
clean_genres    57819
dtype: int64

In [14]:
mov_gen_df = mov_gen_df.drop(columns=["clean_genres"])
mov_gen_df.head()

,movie_id,mg_id,genre_id
0,1,1,1
1,1,2,3
2,1,3,4
3,1,4,5
4,1,5,9


In [15]:
# Set Index to Common Column of movie_id For Easier Join
movies_df = movies_df.set_index("movie_id")
ratings_df = ratings_df.set_index("movie_id")
mov_gen_df = mov_gen_df.set_index("movie_id")
cast_df = cast_df.set_index("movie_id")

In [16]:
data_df = ratings_df.join(movies_df, on="movie_id", how="left", sort=True)
data_df = data_df.join(mov_gen_df, on="movie_id", how="left", sort=True)
data_df = data_df.join(cast_df, on="movie_id", how="left", sort=True)

data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id
movie_id,,,,,,,,
1,96032,2.0,Toy Story,1995.0,81.0,1.0,1.0,31.0
1,96032,2.0,Toy Story,1995.0,81.0,2.0,3.0,31.0
1,96032,2.0,Toy Story,1995.0,81.0,3.0,4.0,31.0
1,96032,2.0,Toy Story,1995.0,81.0,4.0,5.0,31.0
1,96032,2.0,Toy Story,1995.0,81.0,5.0,9.0,31.0


In [17]:
data_df["movie_id"] = data_df.index
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id
movie_id,,,,,,,,,
1,96032,2.0,Toy Story,1995.0,81.0,1.0,1.0,31.0,1
1,96032,2.0,Toy Story,1995.0,81.0,2.0,3.0,31.0,1
1,96032,2.0,Toy Story,1995.0,81.0,3.0,4.0,31.0,1
1,96032,2.0,Toy Story,1995.0,81.0,4.0,5.0,31.0,1
1,96032,2.0,Toy Story,1995.0,81.0,5.0,9.0,31.0,1


In [18]:
data_df = data_df.reset_index(drop=True)
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id
0,96032,2.0,Toy Story,1995.0,81.0,1.0,1.0,31.0,1
1,96032,2.0,Toy Story,1995.0,81.0,2.0,3.0,31.0,1
2,96032,2.0,Toy Story,1995.0,81.0,3.0,4.0,31.0,1
3,96032,2.0,Toy Story,1995.0,81.0,4.0,5.0,31.0,1
4,96032,2.0,Toy Story,1995.0,81.0,5.0,9.0,31.0,1


In [19]:
data_df.count()

user_id     52632659
rating      52632659
name        52474568
year        52474568
runtime     52474568
mg_id       46944058
genre_id    46944058
cast_id     52104091
movie_id    52632659
dtype: int64

In [20]:
data_df["runtime"].describe()

count    5.247457e+07
mean     1.152940e+02
std      2.513737e+01
min      0.000000e+00
25%      9.900000e+01
50%      1.120000e+02
75%      1.270000e+02
max      1.256000e+03
Name: runtime, dtype: float64

In [21]:
counts = data_df["movie_id"].value_counts()
counts.describe()

count     53889.000000
mean        976.686504
std        7150.832342
min           1.000000
25%           2.000000
50%           7.000000
75%          57.000000
max      342345.000000
Name: movie_id, dtype: float64

In [22]:
print(data_df["movie_id"].value_counts())

1         342345
589       321290
1198      317525
590       311262
356       291120
           ...  
135374         1
135370         1
135364         1
135362         1
128401         1
Name: movie_id, Length: 53889, dtype: int64


In [23]:
movie_ids = data_df["movie_id"].unique()

movie_counts = pd.DataFrame(data=counts, index=movie_ids)
movie_counts.head()

,movie_id
1,342345
2,81429
3,31170
4,8967
5,15474


In [24]:
clean_movie_ids = movie_counts.loc[movie_counts["movie_id"] > 1000]
clean_movie_ids["counts"] = clean_movie_ids["movie_id"]
clean_movie_ids["movie_id"] = clean_movie_ids.index
clean_movie_ids = clean_movie_ids.reset_index(drop=True)
clean_movie_ids

C:\Users\Bigge\.conda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Bigge\.conda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,counts
0,1,342345
1,2,81429
2,3,31170
3,4,8967
4,5,15474
...,...,...
4771,177615,1464
4772,177765,2701
4773,179819,2273
4774,180031,1473


In [25]:
clean_movie_ids.head()

,movie_id,counts
0,1,342345
1,2,81429
2,3,31170
3,4,8967
4,5,15474


In [26]:
len(clean_movie_ids["movie_id"])

4776

In [27]:
na = float("Nan")
data_df["new_movie_id"] = ""

def limit_ratings(column, column_2):
    i = -1
    for entry in column:
        i += 1
        if entry in column_2:
            data_df.at[i, "new_movie_id"] = entry
        else:
            data_df.at[i, "new_movie_id"] = na
    
    return data_df.head()
            


In [28]:
limit_ratings(data_df["movie_id"], clean_movie_ids["movie_id"])

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id,new_movie_id
0,96032,2.0,Toy Story,1995.0,81.0,1.0,1.0,31.0,1,1
1,96032,2.0,Toy Story,1995.0,81.0,2.0,3.0,31.0,1,1
2,96032,2.0,Toy Story,1995.0,81.0,3.0,4.0,31.0,1,1
3,96032,2.0,Toy Story,1995.0,81.0,4.0,5.0,31.0,1,1
4,96032,2.0,Toy Story,1995.0,81.0,5.0,9.0,31.0,1,1


In [29]:
data_df.count()

user_id         52632659
rating          52632659
name            52474568
year            52474568
runtime         52474568
mg_id           46944058
genre_id        46944058
cast_id         52104091
movie_id        52632659
new_movie_id    38708614
dtype: int64

In [30]:
data_df = data_df.drop_duplicates(keep="first")
data_df = data_df.dropna()

In [31]:
data_df.count()

user_id         38498133
rating          38498133
name            38498133
year            38498133
runtime         38498133
mg_id           38498133
genre_id        38498133
cast_id         38498133
movie_id        38498133
new_movie_id    38498133
dtype: int64

In [32]:
data_df["year"] = data_df["year"].astype(int)
# data_df["mg_id"] = data_df["mg_id"].astype(int)
data_df["genre_id"] = data_df["genre_id"].astype(int)
data_df["cast_id"] = data_df["cast_id"].astype(int)

data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id,new_movie_id
0,96032,2.0,Toy Story,1995,81.0,1.0,1,31,1,1
1,96032,2.0,Toy Story,1995,81.0,2.0,3,31,1,1
2,96032,2.0,Toy Story,1995,81.0,3.0,4,31,1,1
3,96032,2.0,Toy Story,1995,81.0,4.0,5,31,1,1
4,96032,2.0,Toy Story,1995,81.0,5.0,9,31,1,1


In [33]:
data_df["trimmed_runtime"] = data_df["runtime"].loc[data_df["runtime"] > 59]

In [34]:
data_df.count()

user_id            38498133
rating             38498133
name               38498133
year               38498133
runtime            38498133
mg_id              38498133
genre_id           38498133
cast_id            38498133
movie_id           38498133
new_movie_id       38498133
trimmed_runtime    38412582
dtype: int64

In [35]:
data_df = data_df.dropna()

In [36]:
data_df.count()

user_id            38412582
rating             38412582
name               38412582
year               38412582
runtime            38412582
mg_id              38412582
genre_id           38412582
cast_id            38412582
movie_id           38412582
new_movie_id       38412582
trimmed_runtime    38412582
dtype: int64

In [38]:
data_df = data_df.drop(columns=["new_movie_id", "trimmed_runtime"])
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id
0,96032,2.0,Toy Story,1995,81.0,1.0,1,31,1
1,96032,2.0,Toy Story,1995,81.0,2.0,3,31,1
2,96032,2.0,Toy Story,1995,81.0,3.0,4,31,1
3,96032,2.0,Toy Story,1995,81.0,4.0,5,31,1
4,96032,2.0,Toy Story,1995,81.0,5.0,9,31,1


In [39]:
# data_df.to_csv("ML_Static_Dataframe_full_alt.csv", index=False)

# Machine Learning Model Creation and Testing

In [40]:
# Declare X and y Variable Values From Dataframe
X = data_df.drop(columns=["user_id", "rating", "name"])
y = data_df["rating"]

X.head()

,year,runtime,mg_id,genre_id,cast_id,movie_id
0,1995,81.0,1.0,1,31,1
1,1995,81.0,2.0,3,31,1
2,1995,81.0,3.0,4,31,1
3,1995,81.0,4.0,5,31,1
4,1995,81.0,5.0,9,31,1


In [41]:
# Create Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Decision Tree Regressor

In [42]:
# Create and Fit Model With Features
model = DecisionTreeRegressor(random_state=0)
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [43]:
# Determine Coefficient of Determination (R^2)
model.score(X_test, y_test)

0.18949931054992308

In [44]:
# Predict Rating Values Based on Initial Datatable Features, Populate Dataframe Column
dcsntree_preds = model.predict(X)
data_df["Predicted Rating"] = dcsntree_preds
data_df.head()

,user_id,rating,name,year,runtime,mg_id,genre_id,cast_id,movie_id,Predicted Rating
0,96032,2.0,Toy Story,1995,81.0,1.0,1,31,1,3.887207
1,96032,2.0,Toy Story,1995,81.0,2.0,3,31,1,3.886192
2,96032,2.0,Toy Story,1995,81.0,3.0,4,31,1,3.886407
3,96032,2.0,Toy Story,1995,81.0,4.0,5,31,1,3.884429
4,96032,2.0,Toy Story,1995,81.0,5.0,9,31,1,3.890213


# Export to CSV File For Visualization Software Access

In [45]:
data_df.to_csv("full_df_prediction_dataframe.csv", index=False)